In [5]:
import pandas as pd
import time
import conn_creds as cc
import numpy as np
import requests, datetime, re
from bs4 import BeautifulSoup
import mysql.connector
from sqlalchemy import create_engine
import sqlalchemy
pd.options.mode.chained_assignment = None

# establish MySQL engine with credentials
db_name = 'crypto_price_db'
table_name = 'historical_crypto'
engine = create_engine('mysql+mysqlconnector://'+cc.db_username+':'+cc.db_pass+'@'+cc.db_host+':'+cc.db_port+'/'+db_name, echo=False)

In [ ]:
######## ONLY RUN CELL 2 IF YOU CURRENTLY HAVE NO DATA IN YOUR DATABASE

In [ ]:
### Initial pull: use this to build a database in MySQL to pull all CoinMarketCap (CMC) data. This cell needs to be run the very first time you make a database. 
### After the database is created, run the cells after this.
## Step 1 of 2: Build pandas dataframe of historical coin data.

#build list of each coin
url = 'https://coinmarketcap.com/all/views/all/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html5lib')
names=soup.find_all("a",{"class":"currency-name-container"})
names_strings = [str(i) for i in names]

## pull name from each string in a list
names_list =[]
for x in names_strings:
    names_list.append(re.split('/"|currencies/',x)[-2])

#scrape each coin
initial_CMC_pull = pd.DataFrame()
count = 0
missing_coins = []

for item in names_list:
    count += 1
    try:
        url = 'https://coinmarketcap.com/currencies/' + item + '/historical-data/?start=20100101&end=' + str(datetime.datetime.now())[:10].replace('-','')
        r = requests.get(url)

        soup = BeautifulSoup(r.text, 'lxml')
        table = soup.find('table')

        headers = {'coin_name': [np.nan], 'date_hist': [np.nan], 'open_price': [np.nan], 'high_price': [np.nan], 'low_price': [np.nan], 'close_price': [np.nan], 'volume': [np.nan], 'market_cap': [np.nan]}
        coin_data = pd.DataFrame(headers)

        for row in table.find_all('tr')[1:]:
            col = row.find_all('td')

            coin_name = item
            date = col[0].text.strip()
            open_price = col[1].text.strip()
            high_price = col[2].text.strip()
            low_price = col[3].text.strip()
            close_price = col[4].text.strip()
            volume = col[5].text.strip()
            market_cap = col[6].text.strip()

            temp_df = {'coin_name':coin_name,'date_hist': date, 'open_price': open_price, 'high_price': high_price, 'low_price': low_price, 'close_price': close_price, 'volume': volume, 'market_cap': market_cap}
            coin_data = coin_data.append(temp_df, ignore_index=True)

        coin_data = coin_data.drop(0)
        final_coin_data = final_coin_data.append(coin_data)
        print(str(count) + '. ' + str(item) + '...done')
    except:
        missing_coins.append(item)
        print(str(count)+ '. missing '+ str(item))
        
final_coin_data = cmc_df_cleanup(final_coin_data)
final_coin_data.reset_index(drop=True,inplace=True)

# define datatypes from scrape
dtypes = {'coin_name':sqlalchemy.types.NVARCHAR(length=100),'date_hist':sqlalchemy.types.Date,'open_price':sqlalchemy.types.Float(asdecimal=True),'close_price':sqlalchemy.types.Float(asdecimal=True),
          'high_price':sqlalchemy.types.Float(asdecimal=True),'low_price':sqlalchemy.types.Float(asdecimal=True),'market_cap':sqlalchemy.types.BIGINT,'volume':sqlalchemy.types.BIGINT}

# load data to MySQL
final_coin_data.to_sql(name=table_name, con=engine, if_exists = 'replace', index=False, chunksize=20000 , dtype=dtypes)

In [ ]:
###### RUN BELOW TWO CELLS TO UPDATE WITH NEW COINS ON CMC AND UPDATE NEW DATA FOR RECENT DATES

In [6]:
### Define functions for data cleanup, pulling new coins from CMC, and pulling new data for existing coins in the database from CMC
count = 0

## Function for cleaning up df before MySQL push

def cmc_df_cleanup(df):
    try:
        # convert all random characters in string varaibles into nan
        df = df[['coin_name','date_hist','open_price','close_price','high_price','low_price','market_cap','volume']]
        df[['open_price','close_price','high_price','low_price','market_cap','volume']] = (
        df[['open_price','close_price','high_price','low_price','market_cap','volume']].apply(lambda x: x.str.replace(',','')))
        df[['open_price','close_price','high_price','low_price','market_cap','volume']] = (
        df[['open_price','close_price','high_price','low_price','market_cap','volume']].apply(lambda x: x.replace('-|^\s*$',np.nan,regex=True)))
        # convert to datetime
        df['date_hist'] = pd.to_datetime(df['date_hist'])
        return df
    except:
        pass

## Function for updating missing historical data in CMC

def pull_missing_dates_cmc(coin,start_date):
    try:
        updated_cmc = pd.DataFrame()
        url = 'https://coinmarketcap.com/currencies/' + coin + '/historical-data/?start=' + start_date.replace('-','') + "&end=" + str(datetime.datetime.now())[:10].replace('-','')

        r = requests.get(url)

        soup = BeautifulSoup(r.text, 'lxml')
        table = soup.find('table')

        headers = {'coin_name': [np.nan], 'date_hist': [np.nan], 'open_price': [np.nan], 'high_price': [np.nan], 'low_price': [np.nan], 'close_price': [np.nan], 'volume': [np.nan], 'market_cap': [np.nan]}
        coin_data = pd.DataFrame(headers)

        for row in table.find_all('tr')[1:]:
            col = row.find_all('td')

            coin_name = coin
            date = col[0].text.strip()
            open_price = col[1].text.strip()
            high_price = col[2].text.strip()
            low_price = col[3].text.strip()
            close_price = col[4].text.strip()
            volume = col[5].text.strip()
            market_cap = col[6].text.strip()

            temp_df = {'coin_name':coin_name,'date_hist': date, 'open_price': open_price, 'high_price': high_price, 'low_price': low_price, 'close_price': close_price, 'volume': volume, 'market_cap': market_cap}
            coin_data = coin_data.append(temp_df, ignore_index=True)

        coin_data = coin_data.drop(0)
        updated_cmc = updated_cmc.append(coin_data)
        return updated_cmc
    except:
        pass

## Function for updating database with new coins on CMC
        
def pull_new_coins(coin):
    blank_df = pd.DataFrame()
    try:
        final_coin_data = pd.DataFrame()
        url = 'https://coinmarketcap.com/currencies/' + coin + '/historical-data/?start=20100101&end='  + str(datetime.datetime.now())[:10].replace('-','')

        r = requests.get(url)

        soup = BeautifulSoup(r.text, 'lxml')
        table = soup.find('table')

        headers = {'coin_name': [np.nan], 'date_hist': [np.nan], 'open_price': [np.nan], 'high_price': [np.nan], 'low_price': [np.nan], 'close_price': [np.nan], 'volume': [np.nan], 'market_cap': [np.nan]}
        coin_data = pd.DataFrame(headers)

        for row in table.find_all('tr')[1:]:
            col = row.find_all('td')

            coin_name = coin
            date = col[0].text.strip()
            open_price = col[1].text.strip()
            high_price = col[2].text.strip()
            low_price = col[3].text.strip()
            close_price = col[4].text.strip()
            volume = col[5].text.strip()
            market_cap = col[6].text.strip()

            temp_df = {'coin_name':coin_name,'date_hist': date, 'open_price': open_price, 'high_price': high_price, 'low_price': low_price, 'close_price': close_price, 'volume': volume, 'market_cap': market_cap}
            coin_data = coin_data.append(temp_df, ignore_index=True)

        coin_data = coin_data.drop(0)
        final_coin_data = final_coin_data.append(coin_data)
        count += 1
        return final_coin_data
    except:
        return blank_df

In [8]:
%%time
### Coin Data Update.
## The code below will update MySQL with new data for each coin and will identify any new coins that have been added to CMC


## Find the last date of available data of the coins in MySQL
# pull data from mysql
miss_hist_dates = pd.DataFrame
miss_hist_dates = pd.read_sql('select coin_name, max(date_hist) from historical_crypto group by coin_name;',con=engine)

## Create list of any missing coins in MySQL but found in CMC
# Create list of each coin on CMC
url = 'https://coinmarketcap.com/all/views/all/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html5lib')
names=soup.find_all("a",{"class":"currency-name-container"})
names_strings = [str(i) for i in names]

## pull name from each string in a list
CMC_coins =[]
for x in names_strings:
    CMC_coins.append(re.split('/"|currencies/',x)[-2])

missing_coins_mysql = pd.read_sql('select distinct coin_name from historical_crypto;',con=engine)
missing_coins_mysql = missing_coins_mysql.values.tolist()
missing_coins_mysql = [item for sublist in missing_coins_mysql for item in sublist]

missing_coins = (set(CMC_coins).difference(missing_coins_mysql))

## script to pull missing coins
## Pull any new coins from CMC that are not in MySQL

new_coins = pd.DataFrame()
count = 0
for coin in missing_coins:
    if not missing_coins:
        pass
    else:
        new_coins = new_coins.append(pull_new_coins(coin))
        
    
if count == 0 or count >= 2:
    print(str(count) + ' new coins collected')
elif count == 1:
    print(str(count) + ' new coin collected')

new_coins = cmc_df_cleanup(new_coins)
    
### Pull missing historical values for coins already in MySQL

updated_coins = pd.DataFrame()
count = 0
for item in miss_hist_dates.values.tolist()[:5]: 
    if str(item[1]).replace('-','') == str(datetime.datetime.now())[:10].replace('-',''):
        pass
    else:
        count += 1
        updated_coins = updated_coins.append(pull_missing_dates_cmc(item[0],str(item[1] + datetime.timedelta(days=1)).replace('-','')))
        
if count == 0 or count >= 2:
    print(str(count) + ' coins updated with data up to ' + str(datetime.datetime.now())[:10])
elif count == 1:
    print(str(count) + ' coin updated with data up to ' + str(datetime.datetime.now())[:10])


updated_coins = cmc_df_cleanup(updated_coins)

try:
    new_and_update = pd.concat([updated_coins,new_coins])
    new_and_update.reset_index(drop=True,inplace=True)
except:
    pass

# load data to MySQL
new_and_update.to_sql(name=table_name, con=engine, if_exists = 'append', index=False, chunksize=20000)

0 new coins collected
5 coins updated with data up to 2018-05-11
Wall time: 3min 6s


In [ ]:
########## STILL NOT WORKING BELOW

In [30]:
analysis_historical['close_price'].astype(flt6)

In [52]:
analysis_historical =final_coin_data[['coin_name','date_hist','open_price','close_price']]

a['Volitility_30_day'] = a['close_price'].rolling(window=30).std().reset_index(drop=True)
#analysis_historical['Volitility_90_day'] = analysis_historical.groupby('coin_name')['close_price'].rolling(window=90).std().reset_index(drop=True)
#analysis_historical['Volitility_365_day'] = analysis_historical.groupby('coin_name')['close_price'].rolling(window=365).std().reset_index(drop=True) 

## fyi, normally annual volitility is 255 instead of 365 since there are only 255 trading days in a year, but since crypto is all 365 then I leave at 365

In [19]:
analysis_historical.to_sql(name='analysis_historical', con=engine, if_exists = 'replace', index=False, chunksize=10000)

In [42]:
# sort dates in descending order
analysis_historical.groupby('coin_name').apply(lambda x: x.sort_index(ascending=False, inplace=True))

# calculate daily logarithmic return
analysis_historical['daily_log_return'] = (np.log(analysis_historical['close_price'] /
    analysis_historical['close_price'].shift(-1)))





In [51]:
a = analysis_historical[analysis_historical['coin_name']=='bitcoin']